In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

In [2]:
#Hallo Herr Zimmermann,

 

#vielen Dank für Ihre Nachricht.

#Ich habe mir das Notebook angeschaut und eine Lösung vorgeschlagen. Allerdings verstehe ich das Ziel der Methode nicht ganz, von daher müssten Sie einmal prüfen, ob ich es richtig interpretiert habe.

#Mein Vorschlag kann folgendes: für ein Fenster von z.B. 10 Nachrichten vorher oder nachher (Achtung: nachher ist für eine Prediction nicht verwendbar! Zukunftsinformation nennt man das) wird gezählt, wie oft die severity-Klassen 1, 2 und 3 vorkommen. Dies lege ich in drei Spalten ab: pre_no_srv_1, pre_no_srv_2, pre_no_srv_3

#Die vielen Spalten, die durch das „Shiften“ zustande kommen, können Sie auch wieder löschen. Diese sind nur für den Zwischenschritt nötig.

#So bewege ich mich dauerhaft in der Pandas-Welt und von der Performance (Laufzeit und Speicher) gibt es keine Problem.

 

#Ist es das, was Sie erreichen wollten?

 

#Hinsichtlich der Mappings habe ich Ihnen auch noch einen Vorschlag gemacht.

 

#Viele Grüße und ein schönes Wochenende

#Peer Küppers 

In [3]:
#Daten einlesen
df_orig = pd.read_parquet("D:/Eclipse Python/DataScience/res/event_ano.parquet.gzip")

In [4]:
df = df_orig.set_index("event_timestamp")
del df['Unnamed: 0']
df.sort_index(inplace=True)

In [5]:
line_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_line.unique())]:
    line_mapping[k] = v

df["an_line_no"] = df.an_line.map(line_mapping)  

In [6]:
cell_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_cell.unique())]:
    cell_mapping[k] = v

df["an_cell_no"] = df.an_cell.map(cell_mapping)  

In [7]:
robot_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_robot.unique())]:
    robot_mapping[k] = v

df["an_robot_no"] = df.an_robot.map(robot_mapping)  

In [8]:
message_severity_mapping = {"Error":1,"Warning":2,"Information":3}

df["message_severity"] = df.message_severity.apply(lambda x: message_severity_mapping[x] if x in message_severity_mapping.keys() else x)

In [9]:
def dfExtractMethod(df,extract_value):
    df_extract = df.loc[df['an_line_no'] == extract_value]
    return df_extract

In [10]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,no_of_messages):
    dataFrame = pd.DataFrame()
    
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
                
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
        
        dataFrame = dataFrame.append(df_extract)
    return dataFrame

In [11]:
df_ErrorsPerLinesPattern = dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,10)
df_ErrorsPerLinesPattern.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_all.parquet.gzip", compression='gzip')

In [12]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
    
        df_extract.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_single_{}.parquet.gzip".format(j),compression='gzip')

In [13]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,10)

In [37]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetLinesMethod(df,no_of_messages,extract_value):
    for i in range(0,28):
        df_extract = dfExtractMethod(df,i)
        df_extract = df_extract.reset_index()
        
        number = 0
        size = df_extract.size
        data1 = []
        data2 = []
        
        for j in df_extract:
            data2.append(j)

        print(data2)
        data1.append(data2)
            
        for k in df_extract.values:
            if(k[3] == extract_value):
                data3 = []
                data4 = []
                for l in range(1,11):
                    if(number+l <= size and number+l >= 0):
                        data3.append(df_extract.values[number-l])
                        data4.append(k)

                    if(l == 10):
                        data1.append(data3)
                        data1.append(data4[0])
            else:
                number = number + 1
                continue
        
        print("Fertig{}".format(i))
        myErrorsPerLinesPatternArray = np.asarray(data1)
        df_ErrorsPerLinesPattern = pd.DataFrame(myErrorsPerLinesPatternArray)
        print(df_ErrorsPerLinesPattern)
        df_ErrorsPerLinesPattern.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_DataSet_per_line_line{}.parquet.gzip".format(i), compression='gzip')
    print("Fertig")

In [38]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetLinesMethod(df,10,1)

['event_timestamp', 'message_number', 'message_category', 'message_severity', 'an_title', 'an_description', 'an_line', 'an_cell', 'an_robot', 'an_line_no', 'an_cell_no', 'an_robot_no']
Fertig0
                                                     0
0    [event_timestamp, message_number, message_cate...
1    [[2016-06-30 12:37:36, 50024, Motion, 2, None,...
2    [2016-06-30 12:40:14, 71201, IOCommunication, ...
3    [[2016-06-30 12:40:14, 71201, IOCommunication,...
4    [2016-06-30 12:40:14, 71201, IOCommunication, ...
5    [[2016-06-30 12:40:15, 10150, Operational, 3, ...
6    [2016-06-30 12:40:21, 134010, Paint, 1, None, ...
7    [[2016-06-30 12:40:18, 10129, Operational, 3, ...
8    [2016-06-30 12:40:22, 134010, Paint, 1, None, ...
9    [[2016-06-30 16:38:47, 10052, Operational, 3, ...
10   [2016-06-30 16:39:49, 71201, IOCommunication, ...
11   [[2016-06-30 16:38:47, 10052, Operational, 3, ...
12   [2016-06-30 16:39:49, 71201, IOCommunication, ...
13   [[2016-06-30 16:39:50, 10010, Op

ValueError: parquet must have string column names

In [37]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetAllLinesMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            if(df_extract.message_severity == 1):
                
        
        #df_extract.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_DataSet.parquet.gzip".format(j), compression='gzip')

In [ ]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetAllLinesMethod(df,10)